In [73]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [74]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'  

In [75]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
import glob 
import os   

os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR/cs_mods/July2023') # set cwd
model_list = []
for file in glob.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file[:-4] # remove .xml 
    model.name = file[:-4] # remove .xml
    model_list.append(model)
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') # reset cwd

No objective coefficients in model. Unclear what should be optimized


In [ ]:
# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

[<Model Human-GEM_28062023 at 0x1deb905f8e0>,
 <Model mod_RPE_PRcone at 0x1debf653580>,
 <Model mod_RPE_PRcone_Liang at 0x1dec6e092d0>,
 <Model mod_RPE_PRcone_Liang_singlePR at 0x1dece49c670>,
 <Model mod_RPE_PRcone_Liang_singleRPE at 0x1ded23273a0>,
 <Model mod_RPE_PRcone_Lukowski at 0x1ded72cee60>,
 <Model mod_RPE_PRcone_Lukowski_singlePR at 0x1dedf5aed70>,
 <Model mod_RPE_PRcone_Lukowski_singleRPE at 0x1dee1bf2770>,
 <Model mod_RPE_PRcontrol at 0x1dee5d2fd60>,
 <Model mod_RPE_PRrod at 0x1df012bc4f0>,
 <Model mod_RPE_PRrod_Liang at 0x1df0b83ee60>,
 <Model mod_RPE_PRrod_Liang_singlePR at 0x1df0bd307c0>,
 <Model mod_RPE_PRrod_Liang_singleRPE at 0x1df14737c70>,
 <Model mod_RPE_PRrod_Lukowski at 0x1df1a06f6a0>,
 <Model mod_RPE_PRrod_Lukowski_singlePR at 0x1df20c05270>,
 <Model mod_RPE_PRrod_Lukowski_singleRPE at 0x1df2491be20>,
 <Model Human1 at 0x1de4c01f970>,
 <Model RPE_PR at 0x1de69755330>]

In [ ]:
# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)

for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

In [ ]:
# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)

In [ ]:
# dark new estimates
# MAR09048 = oxygen exchange
# MAR09034 = glucose exchange
# MAR09135 = lactate exchange

def set_bound(model, ex_dict): 
    """set bounds for exchange reactions in model based on dict"""
    for ex in ex_dict.keys():
        if ex + '_RPE' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_RPE').bounds = ex_dict[ex]
        elif ex + '_PR' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_PR').bounds = ex_dict[ex]
        elif ex + '_eRPE_PR' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_eRPE_PR').bounds = ex_dict[ex]
        elif ex in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex).bounds = ex_dict[ex]
        else:  
            print('no exchange reaction for ' + ex + ' in model: ' + model.id)
    return model


ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}

for m in model_list:
    set_bound(m, ex_bounds)

results_dict = {}
for m in model_list:
    m.optimize()
    results_dict[m.id] = m.objective.value
pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T


,max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,24.7325
mod_RPE_PRcone,23.9000
mod_RPE_PRcone_Liang,23.9000
mod_RPE_PRcone_Liang_singlePR,11.1350
mod_RPE_PRcone_Liang_singleRPE,23.9000
mod_RPE_PRcone_Lukowski,24.7325
mod_RPE_PRcone_Lukowski_singlePR,22.2350
mod_RPE_PRcone_Lukowski_singleRPE,23.9000
mod_RPE_PRcontrol,24.7325
mod_RPE_PRrod,23.9000


In [ ]:
# get model from list with particular id
def get_mod_by_id(item_list, id):
    for item in item_list:
        if item.id == id:
            return item
m = get_mod_by_id(model_list, 'Human-GEM_28062023')

from src.get_info import make_rxn_df

rxns=make_rxn_df(m)
rxns.to_clipboard()
rxns

,cell,lb,ub,sbo,bigg.reaction,reactome,rhea,ec-code,metanetx.reaction,kegg.reaction,tcdb,vmhreaction,name,subsystem,reaction,met_IDs,GPR
MAR03905,None,0.0,1000.0,SBO:0000176,ALCD2x,,"[25291, 25290]","[1.1.1.1, 1.1.1.71]",MNXR95725,R00754,,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c] + NAD+[c] --> acetaldehyde[c] + H+[...,MAM01796c + MAM02552c --> MAM01249c + MAM02039...,ENSG00000147576 or ENSG00000172955 or ENSG0000...
MAR03907,None,0.0,1000.0,SBO:0000176,ALCD2y,,,1.1.1.2,MNXR95726,R00746,,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c] + NADP+[c] --> acetaldehyde[c] + H+...,MAM01796c + MAM02554c --> MAM01249c + MAM02039...,ENSG00000117448
MAR04097,None,0.0,1000.0,SBO:0000176,ACS,,"[23177, 23176]",6.2.1.1,MNXR95413,R00235,,ACS,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,acetate[c] + ATP[c] + CoA[c] --> acetyl-CoA[c]...,MAM01252c + MAM01371c + MAM01597c --> MAM01261...,ENSG00000131069
MAR04099,None,0.0,1000.0,SBO:0000176,ACSm,,"[23177, 23176]",6.2.1.1,MNXR95413,R00235,,ACSm,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,acetate[m] + ATP[m] + CoA[m] --> acetyl-CoA[m]...,MAM01252m + MAM01371m + MAM01597m --> MAM01261...,ENSG00000111058 or ENSG00000154930
MAR04108,None,0.0,1000.0,SBO:0000176,,,,6.2.1.1,MNXR105304,R00236,,r0068,acetyl adenylate:CoA acetyltransferase,Glycolysis / Gluconeogenesis,acetyl adenylate[c] + CoA[c] --> acetyl-CoA[c]...,MAM01257c + MAM01597c --> MAM01261c + MAM01334...,ENSG00000131069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR20107,None,0.0,1000.0,SBO:0000176,"[HBZOPT10m, R_HBZOPT10m]",,44564,2.5.1.39,MNXR100567,,,,MAR20107,Ubiquinone and other terpenoid-quinone biosynt...,4-hydroxybenzoate[r] + all-trans-decaprenyl-di...,MAM00995r + MAM01316r --> MAM00767r + MAM02759r,ENSG00000120942
MAR20108,None,0.0,1000.0,SBO:0000176,"[HBZOPT10m, R_HBZOPT10m]",,44564,2.5.1.39,MNXR100567,,,,MAR20108,Ubiquinone and other terpenoid-quinone biosynt...,4-hydroxybenzoate[g] + all-trans-decaprenyl-di...,MAM00995g + MAM01316g --> MAM00767g + MAM02759g,ENSG00000120942
MAR20109,None,0.0,1000.0,SBO:0000176,"[HBZOPT10m, R_HBZOPT10m]",,44564,2.5.1.39,MNXR100567,,,,MAR20109,Ubiquinone and other terpenoid-quinone biosynt...,4-hydroxybenzoate[n] + all-trans-decaprenyl-di...,MAM00995n + MAM01316n --> MAM00767n + MAM02759n,ENSG00000120942
MAR20110,None,0.0,1000.0,SBO:0000176,,,"[15642, 15641]",,MNXR102097,,,,MAR20110,Miscellaneous,H+[m] + OAA[m] --> CO2[m] + pyruvate[m],MAM02039m + MAM02633m --> MAM01596m + MAM02819m,ENSG00000180185


In [ ]:
m.reactions.get_by_id('MAR03964')


Reaction identifier,MAR03964
Name,ATP phosphohydrolase
Memory address,0x1debcfd1300
Stoichiometry,MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [ ]:

for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

In [ ]:
def fba_analysis(mod, boundary_dicts, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    conditions = dict()
    ovs = dict() # objective values
    f = dict() # fba fluxes
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 0
        
    with mod as m:
            
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # objective values
            ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
            
            if fba.status == 'optimal':

                # write analysis info / results into dicts

                # uptake / secretion
                uptake_summary = m.summary().uptake_flux
                secretion_summary = m.summary().secretion_flux
                uptake[i] = pd.DataFrame(uptake_summary['flux'])
                secretion[i] = pd.DataFrame(secretion_summary['flux'])
                
                # all fluxes
                f[i] = fba.to_frame()['fluxes']
          
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)  
    
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    
    ovs_df  = pd.concat([pd.DataFrame(boundary_dicts),\
                         pd.DataFrame(boundary_labels),\
                          pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T],axis=1)
    
    # uptake / secretion dfs

    uptake_df = pd.concat(uptake,axis=1)
    secretion_df = pd.concat(secretion,axis=1)
    
    secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in secretion_df.index],\
                               columns=['metabolite'],index= secretion_df.index)
    uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in uptake_df.index],\
                               columns=['metabolite'],index= uptake_df.index)
    
    uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
    secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                     
    # sort uptake / secretion dfs
    uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
    secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('fba_analysis_' + results_name + '_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
        
    return [ovs_df, uptake_df, secretion_df, fluxes_df]

In [ ]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set bounds for blood exchange reactions
blood_ratio = {df['id'][i]:df['ratio [glucose]/[x]'] [i] for i in range(len(df['id']))}
glucose_level = -2.622342019
# blood_ratio.update((x, y*glucose_level) for x, y in blood_ratio.items())
blood_ratio

{'MAR09034_RPE': 1.0,
 'MAR09063_RPE': 0.12618432385874245,
 'MAR09061_RPE': 0.090869939707149,
 'MAR09067_RPE': 0.05555555555555555,
 'MAR09041_RPE': 0.050818260120585705,
 'MAR09046_RPE': 0.05017226528854436,
 'MAR09068_RPE': 0.0409130060292851,
 'MAR09044_RPE': 0.03574504737295435,
 'MAR09069_RPE': 0.03229974160206718,
 'MAR09040_RPE': 0.026485788113695088,
 'MAR09038_RPE': 0.023040482342807925,
 'MAR09066_RPE': 0.017226528854435832,
 'MAR09064_RPE': 0.016731266149870805,
 'MAR09043_RPE': 0.01309216192937123,
 'MAR09065_RPE': 0.01119724375538329,
 'MAR09045_RPE': 0.009474590869939707,
 'MAR09071_RPE': 0.009323858742463394,
 'MAR09062_RPE': 0.008828596037898364,
 'MAR09042_RPE': 0.005383290267011197,
 'MAR09070_RPE': 0.0032730404823428077,
 'MAR11961_RPE': 0.00014857881136950905,
 'MAR09039_RPE': 0.014233419465977604}

In [ ]:
def open_blood_exchange(mod,br, glucose_level):  

    """
    Parameters
    ----------
    mod : cobra model
        model to be modified
    br : dict
        dictionary with blood exchange reaction ids as keys and fluxes as values
    glucose_level : float
        glucose level in blood

    Returns
    ------- 
    None
    """
    br_updated = br.copy()
    br_updated.update((x, y * glucose_level) for x, y in br_updated.items())
    for key in br:
        mod.reactions.get_by_id(key).bounds = (br_updated[key],1000)# allow influx for RPE exchange reactions (blood exchange)
        
def close_blood_exchange(mod,br):  
    """
    Parameters
    ----------
    mod : cobra model
        model to be modified
    br : dict
        dictionary with blood exchange reaction ids as keys and fluxes as values
    """
    for key in br:
        mod.reactions.get_by_id(key).bounds = (0,1000)# allow influx for RPE exchange reactions (blood exchange)

In [ ]:
model_list[6]


Name,mod_RPE_PRcone_Lukowski_singlePR
Memory address,1dedf5aed70
Number of metabolites,5180
Number of reactions,6335
Number of genes,2206
Number of groups,149
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [ ]:
from src.get_info import make_rxn_df
# make df with rxns in model
rxns = make_rxn_df(model_list[4])
rxns.to_clipboard()
rxns

,cell,lb,ub,sbo,bigg.reaction,reactome,rhea,ec-code,metanetx.reaction,kegg.reaction,tcdb,vmhreaction,name,subsystem,reaction,met_IDs,GPR
MAR03905_RPE,RPE,0.0,1000.0,SBO:0000176,ALCD2x,,"[25291, 25290]","[1.1.1.1, 1.1.1.71]",MNXR95725,R00754,,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NAD+[c_RPE] --> acetaldehyde[...,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ENSG00000147576 or ENSG00000172955 or ENSG0000...
MAR03907_RPE,RPE,0.0,1000.0,SBO:0000176,ALCD2y,,,1.1.1.2,MNXR95726,R00746,,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NADP+[c_RPE] --> acetaldehyde...,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ENSG00000117448
MAR04281_RPE,RPE,-1000.0,1000.0,SBO:0000176,,,23444,1.1.1.27,MNXR101040,R00703,,r0173,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[x_RPE] + NADH[x_RPE] + pyruvate[x_RPE] <=> ...,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,ENSG00000111716 or ENSG00000134333
MAR04388_RPE,RPE,-1000.0,1000.0,SBO:0000176,LDH_L,,23444,1.1.1.27,MNXR101040,R00703,,LDH_L,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[c_RPE] + NADH[c_RPE] + pyruvate[c_RPE] <=> ...,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04283_RPE,RPE,0.0,1000.0,SBO:0000176,,,"[25299, 25298]",1.2.1.5,MNXR95750,R00711,,ALDD2y,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,acetaldehyde[c_RPE] + H2O[c_RPE] + NADP+[c_RPE...,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,ENSG00000006534 or ENSG00000108602 or ENSG0000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR10132_eRPE_PR,PR,0.0,1000.0,SBO:0000627,,,,,,,,,,Exchange/demand reactions,tauro-hyodeoxycholic acid[e_RPE_PR] -->,MAM10041e_RPE_PR -->,
MAR08583_eRPE_PR,PR,0.0,1000.0,SBO:0000176,AMY1e,,,3.2.1.3,"[MNXR107171, MNXR95832]",R01790,,AMY1e,"1,4-alpha-D-Glucan glucohydrolase",Starch and sucrose metabolism,8.0 H2O[e_RPE_PR] + starch structure 1[e_RPE_P...,8.0 MAM02040e_RPE_PR + MAM02936e_RPE_PR --> 8....,ENSG00000257335
MAR08487_eRPE_PR,PR,0.0,1000.0,SBO:0000176,NTD2e,,"[29360, 29359]",3.1.3.5,MNXR102032,R00963,,NTD2e,uridine 5'-monophosphate phosphohydrolase,Nucleotide metabolism,H2O[e_RPE_PR] + UMP[e_RPE_PR] --> Pi[e_RPE_PR]...,MAM02040e_RPE_PR + MAM03114e_RPE_PR --> MAM027...,ENSG00000122643 or ENSG00000135318
MAR03042_eRPE_PR,PR,-1000.0,1000.0,SBO:0000176,RE2269E,,,3.4.21.39,MNXR103643,,,RE2269E,Chymase,Miscellaneous,2.0 H2O[e_RPE_PR] + kinetensin[e_RPE_PR] <=> k...,2.0 MAM02040e_RPE_PR + MAM02318e_RPE_PR <=> MA...,ENSG00000092009


In [ ]:
# set objective function (ATP hydrolysis in PR for combined models)

for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)


In [ ]:
# modify combined models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964_PR' # ATP demand in PR
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) # close all exchange reactions
    open_blood_exchange(m,blood_ratio,glucose_level)# allow influx for RPE exchange reactions (blood exchange)

c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [ ]:
# modify single models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964' # ATP demand in PR
    open_blood_exchange(m,blood_ratio,glucose_level)# allow influx for RPE exchange reactions (blood exchange)

ValueError: invalid objective

In [ ]:
from src.analysis import create_permutation_dicts
# MAR09048_RPE = oxygen exchange
# MAR09034_RPE = glucose exchange
# MAR09135_RPE = lactate exchange
# MAR04896_PR_RPE =  O2[c_PR] <=> O2[e_RPE_PR]

dark_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.2)'],\
                      'glucose':['limited glucose (2.6)'],\
                      'lactate':['lactate efflux > 4.53'],\
                      'oxygen RPE -> PR':['fixed (1.9)','unlimited']})
dark_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.2,1000)],\
                      'MAR09034_RPE':[(-2.62,1000)],\
                      'MAR09135_RPE':[(4.53,1000)],\
                      'MAR04896_PR_RPE':[(-1.9,-1.9),(-1000,1000)]})

dark_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (3.51)'],\
                      'glucose':['limited glucose (4.18)'],\
                      'lactate':[ 'lactate efflux > 7.25'],\
                      'oxygen RPE -> PR':['fixed (3.21)','unlimited']})
dark_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-3.51,1000)],\
                      'MAR09034_RPE':[(-4.18,1000)],\
                      'MAR09135_RPE':[(7.25,1000)],\
                      'MAR04896_PR_RPE':[(-3.21,-3.21),(-1000,1000)]})

light_old_labels = create_permutation_dicts({'oxygen':['limited oxygen (1.84)'],\
                      'glucose':['limited glucose (1.37)'],\
                      'lactate':['lactate efflux > 2.81'],\
                      'oxygen RPE -> PR':['fixed (1.59)','unlimited']})
light_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(-1.84,1000)],\
                      'MAR09034_RPE':[ (-1.37,1000)],\
                      'MAR09135_RPE':[ (2.81,1000)],\
                      'MAR04896_PR_RPE':[(-1.59,-1.59),(-1000,1000)]})

light_new_labels = create_permutation_dicts({'oxygen':['limited oxygen (2.61)'],\
                      'glucose':['limited glucose (1.93)'],\
                      'lactate':['lactate efflux > 2.95'],\
                      'oxygen RPE -> PR':['fixed (2.31)','unlimited']})
light_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(-2.61,1000)],\
                      'MAR09034_RPE':[ (-1.93,1000)],\
                      'MAR09135_RPE':[ (2.95,1000)],\
                      'MAR04896_PR_RPE':[(-2.31,-2.31),(-1000,1000)]})

In [ ]:
# FVA



In [ ]:
# SAMPLE